# Robot stuff

In this notebook we can check how to sample the trajectory of the robot, how to get velocity profiles from the trajectory, which can be used to control the robot, etc.  

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

### Trajectory sampled uniformly in time

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import EPSILON, ROBOT_WIDTH, MM
from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_complexity = 5 #complexity of trajectory
dim = 2 # dimension of setup. 
n_positions = 10
# model = 'full_bandlimited'
model = 'polynomial'
# model = 'bandlimited'

trajectory = Trajectory(n_complexity, dim=dim, model=model, full_period=True)
trajectory.set_coeffs(seed=4)
# trajectory.coeffs = np.zeros_like(trajectory.coeffs)
# trajectory.coeffs[0, 1] = 1
# trajectory.coeffs[1, 3] = 1

times_nonunif = trajectory.get_times(n_samples=n_positions)
basis_nonunif = trajectory.get_basis(times=times_nonunif + 1e-10)
sample_points_nonunif = trajectory.get_sampling_points(basis=basis_nonunif)

plt.figure()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
plt.legend();

### Scale the trajectory to a given bounding box

In [ ]:
from matplotlib.patches import Rectangle

box = np.array([2.0, 2.0])
box = trajectory.scale_bounding_box(box, keep_aspect_ratio=False)
fig, ax = plt.subplots(1)
trajectory.plot(basis_nonunif)
p = Rectangle((0, 0), box[0], box[1], linewidth=1, edgecolor='c',facecolor='none')
ax.add_patch(p)
plt.show()

### Trajectory sampled uniformly in distance

In [ ]:
from trajectory import Trajectory
from environment import Environment

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

times_unif, distances, errors = trajectory.get_times_from_distances(
    n_samples=n_positions,
    time_steps=50000,
    plot=True)

print("cumulative error: {}".format(np.linalg.norm(errors)))
basis_unif = trajectory.get_basis(times=times_unif)
sample_points_unif = trajectory.get_sampling_points(basis=basis_unif)

plt.figure()
trajectory.plot(basis_nonunif, color='blue', label='non uniform')
trajectory.plot(basis_unif, color='orange', marker='x', label='uniform')
plt.legend()

### Trajectory sampled at arbitrary distances

In [ ]:
times = np.array([0, 1., 1.2, 2, 2.2])
speed = 1.2

new_times, distances, errors = trajectory.get_times_from_distances(
    arbitrary_distances = times * speed,
    time_steps=50000,
    plot=True)
print("cumulative error: {}".format(np.linalg.norm(errors)))
new_basis = trajectory.get_basis(times=new_times)

plt.figure()
trajectory.plot(new_basis, label='points on trajectory')
plt.legend()

## Compute the curvature at each point. 

For now there is still a problem. Since the points are not resampled in time, there is not only a radial acceleration (the one that makes the robot turn) but also a tangent acceleration. In plot below, the orange lines showing the curvature radius, are therefore oriented in the direction of the curve. 

For now I am not quite sure if this is a problem, or if we can still try to incorporate this in the controls for the robot. 

In [ ]:
fig, ax = plt.subplots()
trajectory.plot(basis_nonunif, color='orange', label='ground truth')
xlims = ax.get_xlim()
ylims = ax.get_ylim()
plt.axis('equal')

radii, unit_tangents, radial_acc = trajectory.get_curvatures(times_nonunif, ax=ax)
ax.set_xlim(xlims)
ax.set_ylim(ylims)
#plt.xlim([-2, 8])
#plt.ylim([-2, 5])

In [ ]:
# TODO(FD) Below needs some refactoring, 
# there is a lot of duplicate code. 

fig, ax = plt.subplots()
trajectory.plot(basis_unif)
plt.axis('equal')

points_left, points_right = trajectory.get_left_and_right_points(times_unif, width=ROBOT_WIDTH, ax=ax)

In [ ]:
left, right, _ = trajectory.get_left_and_right_arcs(min_max_distance=0.1, plot=True)
plt.show()

## Generate trajectories for both wheels 
This function uses milimiters as units!

In [ ]:
import csv

FOLDER = 'controls/'

def generate_control_parameters(trajectory, 
                                fname='', 
                                min_max_distance=50,
                                curvature_decimals=1,
                                plot=True,
                                comand_decimals=2,
                               **kwargs):
    """Generate control files for the robot.
    This function uses milimeters as units of it's imput and output, thats how they should be passed to the robot
    :param fname: where to save parameters and plots. extension is ignored.  
    :param plot: if True, plot and save figure of trajectory.
    :param curvature_decimals, min_max_distance, width: See trajectory.get_left_and_right_arcs for details. 
    :param command_decimals: precision at which to save control parameters.
    """
    left, right, command_times = trajectory.get_left_and_right_arcs(
        curvature_decimals=curvature_decimals,
        min_max_distance=min_max_distance*MM,
        width=ROBOT_WIDTH)
    
    fname_parts = fname.split('.')
    
    fname = fname_parts[0] 
    
    fnamepng = FOLDER + fname + '.png'
    print(fnamepng)
    if plot:
        basis = trajectory.get_basis(times=command_times)

        fig, ax = plt.subplots()
        trajectory.plot(basis, color='orange', label='trajectory')
        
        ax.set_xlabel('x[m]')
        ax.set_ylabel('y[m]')
        xlims = ax.get_xlim()
        ylims = ax.get_ylim()
        plt.axis('equal')

        pos_left, pos_right = trajectory.get_left_and_right_points(times=command_times, ax=ax, width=ROBOT_WIDTH)
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
        ax.legend()
        
        # plot the beginning points in a different color. 
        basis_begin = trajectory.get_basis(times=np.array([0., command_times[0]]))
        print(command_times[0])
        coords_begin = trajectory.get_sampling_points(basis=basis_begin)
        coords_diff = coords_begin[:, 1] - coords_begin[:, 0]
        ax.scatter(*coords_begin[:, 0], color='green', zorder=9)
        ax.arrow(
            coords_begin[0, 0],
            coords_begin[1, 0],
            coords_diff[0],
            coords_diff[1],
            width=0.01, head_width=0.2, head_length=0.1,
            color="green", zorder=10)
        
        plt.title(trajectory.get_name())
        #plt.title("{} robot command[s]".format(len(command_times)))
        plt.savefig(fnamepng, bbox_inches='tight')
        print('wrote plots as', fnamepng)
    
    left = np.round(left / MM, comand_decimals)
    right = np.round(right / MM, comand_decimals)
    
    fnametxt = FOLDER + fname + '.gcode'
    with open(fnametxt, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        #header = ['i', 'left_arc', 'right_arc']
        #writer.writerow(header)
        #line = ['G91']
        #writer.writerow(line)
        for i, (r, l) in enumerate(zip(left, right)):
            #line = [i, r, l]
            line = ['G91', 'X{:.2f}'.format(r), 'Y{:.2f}'.format(l)]
            writer.writerow(line)
            if False: #i % 10 == 0:
                #line = ['M114', '']
                line = ['G4','P0.01']
                writer.writerow(line)
                line = ['OCTO1']
                writer.writerow(line)
        print('wrote parameters as', fnametxt)

### Create path to the beggining of the trajectory

In [ ]:
from matplotlib.patches import Arc
from global_variables import MM

def plot_angle(ax, position, first_angle, second_angle, first_color, second_color, diameter=1):

    ax.arrow(
        position[0],
        position[1],
        np.cos(first_angle),
        np.sin(first_angle),
        width=0.01, head_width=0.2, head_length=0.1,
        color=first_color)
    if second_angle - first_angle < np.pi and second_angle - first_angle > 0:
        arc = Arc(position,
                        height=diameter,
                        width=diameter,
                        theta1=first_angle/np.pi*180,
                        theta2=second_angle/np.pi*180,
                        color=first_color)
    else:
        arc = Arc(position,
                height=diameter,
                width=diameter,
                theta1=second_angle/np.pi*180,
                theta2=first_angle/np.pi*180,
                color=first_color)
    ax.add_patch(arc)
    ax.arrow(
        position[0],
        position[1],
        np.cos(second_angle),
        np.sin(second_angle),
        width=0.01, head_width=0.2, head_length=0.1,
        color=second_color)

def navigate_to_beginning(trajectory, fname, base_position=[0,0], base_angle=np.pi/2, plot=True, width=ROBOT_WIDTH,
                         **kwargs):

    starting_position = trajectory.get_sampling_points(trajectory.get_basis(times=[0]))[:, 0]
    starting_tangents, _, _ = trajectory.get_local_frame(times=[0])
    starting_tangents = starting_tangents[:, 0]
    starting_angle = np.arccos(starting_tangents[0])

    base_movement = starting_position - base_position
    movement_length = np.linalg.norm(base_movement)
    if movement_length == 0:
         movement_angle = base_angle
    else:
        movement_angle = np.arccos(base_movement[0]/movement_length)
    
    # to write:
    left = []
    right = []
    # rotate first angle
    first_rotation = movement_angle - base_angle
    left.append(-first_rotation*width/2)
    right.append(first_rotation*width/2)
    # move to position
    left.append(movement_length)
    right.append(movement_length)
    # rotate second angle
    second_rotation = starting_angle - movement_angle
    left.append(-second_rotation*width/2)
    right.append(second_rotation*width/2)
    
    fnametxt = FOLDER + fname + 'P.gcode'
    with open(fnametxt, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=' ')
        for i, (r, l) in enumerate(zip(left, right)):
            line = ['G91', 'X{:.2f}'.format(r / MM), 'Y{:.2f}'.format(l / MM)]
            writer.writerow(line)
        print('wrote pre-trajectory parameters as', fnametxt)
    
    if plot:
        fig, ax = plt.subplots(1)
        trajectory.plot()
        ax.arrow(
            base_position[0],
            base_position[1],
            base_movement[0],
            base_movement[1],
            width=0.01, head_width=0.2, head_length=0.1,
            color="C1")

        plot_angle(ax, base_position, base_angle, movement_angle, "C2", "C1")
        plot_angle(ax, starting_position, movement_angle, starting_angle, "C1", "C0")

        ax.axis('equal')
        plt.show()

navigate_to_beginning(trajectory, 'test_file', base_position=[-1, 0])

### Chose parameters of the trajectories to save

In [ ]:
trajectories = [
    dict(
        parameters = dict(
            model = 'bandlimited',
            n_complexity = 3,
            seed = 5,
        ),
        controls = dict(
            min_max_distance = 100,
            fname = 'bandlimited1',
            base_position=[-2, -2],
        )
    ), 
    dict(
        parameters = dict(
            model = 'polynomial',  
            n_complexity = 3,
            seed = 1,
        ),
        controls = dict(
            curvature_decimals = 2,
            fname = 'polynomial1'
        )
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            n_complexity = 5,
            seed = 3,
        ),
        controls = dict(
            fname = 'full_bandlimited1',
            min_max_distance = 200,
            base_position=[-2, -2],
        )
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0, 1, 0], [0, 0, 1]]),
            name = 'circle'
        ),
        controls = dict(
            fname = 'circle1',
            base_position=[-2, -2]
        )
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0, 1, 0, 0, 0], [0, 0, 0, 1, 0]]),
            name = 'eight',
        ),
        controls = dict(
            fname = 'eight1.txt',
            min_max_distance = 100,
            base_position=[-2, -2]
        )
    ), 
]
    
for traj in trajectories:
    trajectory = Trajectory(**traj['parameters'])
    navigate_to_beginning(trajectory, **traj['controls'])
    generate_control_parameters(trajectory, **(traj['controls']))

## linear trajectories

In [ ]:
box = np.array([2000., 2000.])

trajectories = [
    dict(
        model = 'poly1',
        name = 'straight line'
    )
]

def generate_linear(model):
    c0 = np.array([0, 0])
    if model == 'poly1':
        c1 = np.array([4, 0])
    if model == 'poly2':
        c1 = np.array([3, 1])
    if model == 'poly3':
        c1 = np.array([2, 2])
    if model == 'poly4':
        c1 = np.array([1, 3])
    if model == 'poly5':
        c1 = np.array([0, 4])
    
    trajectory = Trajectory(model='polynomial', coeffs=np.c_[c0, c1])
    return trajectory

for traj in trajectories:
    trajectory = generate_linear(traj['model']) 
    generate_control_parameters(trajectory, fname=traj['model'])

## bandlimited trajectories

In [ ]:
trajectories = [
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0., 1., 0., 0., 0.2,],
                                     [0., 0., 1., 0.2, 0.,]]),
            name = 'triangle'
            ),
        controls = dict(
            fname = 'triangle1.txt',
            min_max_distance = 100
        ),
        box = np.array([4000, 4000]),
        
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0, 1, 0, 0, 0],
                                     [0, 0, 0, 1, 0]], dtype='float64'),
            name = 'eight'
        ),
        controls = dict(
            fname = 'eight2.txt',
            min_max_distance = 100
        ),
        box = np.array([5000, 2000]),
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0, 1, 0], [0, 0, 1]], dtype='float64'),
            name = 'circle'
        ),
        controls = dict(
            fname = 'circle2.txt',
            min_max_distance = 100
        ),
        box = np.array([5000, 5000]),
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0, 1, 0], [0, 0, 1]], dtype='float64'),
            name = 'circle'
        ),
        controls = dict(
            fname = 'circle3.txt',
            min_max_distance = 100
        ),
        box = np.array([2000, 2000]),
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0., 1., 0., 0., 0., 2, 0.],
                              [0., 0., 1., 0., 0., 0., 2]]),
            name = 'circles'
        ),
        controls = dict(
            fname = 'rounds1.txt',
            min_max_distance = 100
        ),
        box = np.array([4000, 4000]),
        
    ), 
    dict(
        parameters = dict(
            model = 'full_bandlimited',
            coeffs = np.array([[0., 1., 0., 0., 0., 1, 0.],
                              [0., 0., 1., 0., 0., 0., -1]]),
            name = 'clover'
        ),  
        controls = dict(
            fname = 'clover1.txt',
            min_max_distance = 100
        ),
        box = np.array([4000, 4000]),
        
    ), 
]
for traj in trajectories:
    trajectory = Trajectory(**traj['parameters'])
    # Dont strech trajectories that have hand designed coefficients
    trajectory.scale_bounding_box(traj['box'] * MM, keep_aspect_ratio='coeffs' in traj['parameters'])
    generate_control_parameters(trajectory, **(traj['controls']))

In [ ]:
import pickle

with open('controls/robot_trajectores.pkl', 'wb') as f:
    pickle.dump(trajectories, f)